# 实验十一 弗兰克-赫兹实验

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as cnst
import scipy
import csv

def read_csv(filename):
    """读取 csv 文件"""
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        return list(filter(len, reader)) # 去除空行

plt.rc('font', size=9)
# 如果你的电脑上没装有 LaTeX 环境或画图仍遇到问题，注释以下行
matplotlib.use('pgf')
plt.rcParams['text.usetex'] = True

In [ ]:
# 读取数据
data_filename = './data/1.csv'
data = read_csv(data_filename)
header = data[0]
data = np.array(data[1:]).transpose()
# 提取数据
V = data[0].astype(float) # (V)
IP1 = data[1].astype(float) # (nA)
IP2 = data[2].astype(float) # (nA)


In [ ]:
if not IP2.any():
    raise ValueError(f"请先在'{data_filename}'文件中填写你的实验数据。")

In [ ]:
def find_period(ls):
    n = len(ls)//2
    # head, tail, dist = ls[:n], ls[-n:], (len(ls) - n)
    # head, tail, dist = ls[:n], ls[n:2*n], n
    head, tail, dist = ls[-2*n:-n], ls[-n:], n
    p1 = sum(xr - xl for xl, xr in zip(head, tail))/n/dist
    print("    用逐差法求得", p1)

    slope, *_ = scipy.stats.linregress(range(len(ls)), ls)
    print("    用最小二乘法求得", slope)

    return p1, slope

In [ ]:
for i, IP in enumerate([IP1, IP2]):
    idx, _ = scipy.signal.find_peaks(IP)
    print(f"第{i+1}组原始数据的最大值点：")
    print(V[idx][:-1]) # [:-1] 以去除最后一个伪峰
    print("平均间距：")
    find_period(V[idx][:-1])

In [ ]:
import itertools
def batched(iterable, n):
    "Batch data into tuples of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(itertools.islice(it, n))):
        yield batch


In [ ]:
# 这里用多个高斯函数的叠加作模型函数以拟合实验数据
# 你可能会想用其他模型函数拟合
def gaussian(x, A, mu, sigma):
    """A: 与高度正相关；mu: 中心位置；sigma: 与宽度正相关"""
    return A * np.exp(-(x - mu)**2 / (2*sigma**2))

def modal(x, *args):
    """args: [b, A1, mu1, sigma1, A2, mu2, sigma2, ...]"""
    """args 中的第一个参数 b 为纵向偏移，剩余参数三个一组各决定一个高斯函数"""
    return sum(
        gaussian(x, *arg3)
        for arg3 in batched(args[1:], 3)
    ) + args[0]

# 初始参数
p0 = [-120, 40, 15, 1, 60, 26, 1, 80, 37, 1, 100, 48, 1, 120, 59, 1, 140, 70, 1, 160, 81, 1]

In [ ]:
def fit_detail(popt):
    b, *param = popt
    param = sorted(batched(param, 3), key=lambda args: args[1])
    mus = [args[1] for args in param]
    print(mus)
    return find_period(mus)

In [ ]:
out_file = ['./fig/2-g1-IA-UG2K.png', './fig/3-g2-IA-UG2K.png'] # 图片保存路径

U0_m = []

# 拟合与作图
for i, IP in enumerate([IP1, IP2]):
    plt.figure(figsize=(6, 4.6))

    # [20:-14]：拟合时去除头、尾的数据；你可能想要依据你的数据作取舍
    popt, pcov = scipy.optimize.curve_fit(modal, V[20:-14], IP[20:-14], p0)
    popt
    # 10, 85：拟合曲线的作图范围，你可能想要依据你的数据作取舍
    xspc = np.linspace(10, 85, 512)
    yfit = modal(xspc, *popt)
    print(f"第{i+1}组拟合曲线的峰位置：")
    U0_m.extend(fit_detail(popt))
    # plt.plot(xspc, yfit, color='orange', label=r"拟合曲线")
    
    plt.plot(V, IP, 'x-', markersize=3, linewidth=1, label=r"原始数据")

    plt.xlabel(r'$U_{G_2 K}\:/\:{\rm V}$')
    plt.ylabel(r'$I_A\:/\:{\rm mA}$')
    # # 如图例上无法显示汉字，更改下面的字体
    # plt.legend(prop={'family': 'Source Han Sans CN'})
    plt.grid()
    plt.savefig(out_file[i], bbox_inches='tight', dpi=600)
    # plt.show()
    

In [ ]:
U0m = np.mean(U0_m)
U0m

In [ ]:
U0 = 11.7 # (V)
E = (U0m - U0)/U0*100
print("相对误差 E =", E, "%")